In [1]:
import pandas as pd
from tensorflow.keras.layers import Dense, Embedding,GlobalMaxPooling1D
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Embedding

2023-01-20 08:27:14.847890: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-20 08:27:15.373398: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-20 08:27:15.373439: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-20 08:27:16.836341: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [84]:
import spacy

def tokenization(text):
    """Use this function to tokenize text.

    :param text: Text as list
    :type text: list[spacy.tokens.token.Token]
    :return: Tokenized text as list
    :rtype: list[spacy.tokens.token.Token]
    """

    token_list = []
    for doc in text: 
        # iterate over tokens in docs
        for token in doc:
            token_list.append(token)

    return token_list


def stop_word_removal(text): 
    """Use this function to remove stop words. 

    :param text: Tokens to remove stop words from 
    :type text: list[spacy.tokens.token.Token]
    :return: Tokens without stop words
    :rtype: list[spacy.tokens.token.Token]
    """

    token_list_without_stop = []
    # Don't add token to list if stop word
    for token in text:
        if token.is_stop == False: 
            token_list_without_stop.append(token)

    return token_list_without_stop


def punctutation_removal(text): 
    """Use this function to remove punctuation.

    :param text: Tokens to remove punctuation from
    :type text: list[spacy.tokens.token.Token]
    :return: Tokens without punctuation
    :rtype: list[spacy.tokens.token.Token]
    """

    token_list_no_stop_no_punct = []
    # Don't add token to list if punctuation
    for token in text:
        if token.is_punct == False:
            token_list_no_stop_no_punct.append(token)

    return token_list_no_stop_no_punct


def lemmatization(text): 
    """Use this function to lemmatize a given text.

    :param text: Tokens to lemmatize
    :type text: list[spacy.tokens.token.Token]
    :return: lemmatized tokens
    :rtype: list[str]
    """

    token_list_no_stop_no_punct_lemmatized = []
    for token in text: 
        if "\n" not in token.lemma_:
            token_list_no_stop_no_punct_lemmatized.append(token.lemma_)
    return token_list_no_stop_no_punct_lemmatized

def padding(text):
    """Use this function to trim the lyrics to a certain size.

    :param text: lemmatized tokens
    :type text: list[str]
    :return: trimmed lemmas or lemmas + padding
    :rtype: list[str]
    """
    trim_size = 360
    padding_sequence = "<padding>"
    
    if len(text) == trim_size:
        return text
    elif len(text) > trim_size:
        return text[0:359]
    else:
        missing = 360 - len(text)
        helper = [padding_sequence] * missing
        return text + helper



def processing_pipeline(song_data):
    """Use this function to execute the entire processing pipeline on given song data.
    Preprocessing steps:
    - Tokenization
    - Stop word removal
    - Punctuation removal
    - Lemmatization
    - ...

    :param song_data: song data saved in a json file containing song name, artist name and lyrics
    :type song_data: dict
    :return: preprocessed song data
    :rtype: dict
    """

    nlp = spacy.load("en_core_web_sm", disable = ['ner'])
    
    for row in range(len(song_data)):
        text_nlp_pipe = list(nlp.pipe([song_data.iloc[row]["Lyrics"]]))
    
        # Tokenization
        song_data.at[row,"Lyrics"] = tokenization(text_nlp_pipe)
        # Stop word removal
        song_data.at[row,"Lyrics"] = stop_word_removal(song_data.iloc[row]["Lyrics"])
        # Punctuation removal
        song_data.at[row,"Lyrics"] = punctutation_removal(song_data.iloc[row]["Lyrics"])
        # Lemmatization
        song_data.at[row,"Lyrics"] = lemmatization(song_data.iloc[row]["Lyrics"])
        # Padding Funktion
        song_data.at[row,"Lyrics"] = padding(song_data.iloc[row]["Lyrics"])
        
    return song_data


In [85]:
lyrics_df = pd.read_csv('./data/lyrics-data.csv')
artists_df = pd.read_csv('./data/artists-data.csv')

In [86]:
english_songs = lyrics_df["language"] == "en"
english_songs_df = lyrics_df[english_songs]
english_songs_df.head(5)
#drop SLink Column since it has no use
english_songs_df=english_songs_df.drop('SLink', axis=1)
# rename columns to more expressive names
english_songs_df= english_songs_df.rename(columns={'ALink': 'Link', 'SName': 'Song_Name', 'Lyric': 'Lyrics', 'language': 'Language'})
# Get artist names from dataset 2
english_songs_df = pd.merge(english_songs_df, artists_df[['Link','Artist']], on='Link', how='left')
english_songs_df = english_songs_df.drop(["Link"], axis=1)

In [87]:
test = english_songs_df[:20]
test.head(2)
list(test["Lyrics"].head(2))

["I feel so unsure\nAs I take your hand and lead you to the dance floor\nAs the music dies, something in your eyes\nCalls to mind a silver screen\nAnd all those sad goodbyes\n\nI'm never gonna dance again\nGuilty feet have got no rhythm\nThough it's easy to pretend\nI know you're not a fool\n\nShould've known better than to cheat a friend\nAnd waste the chance that I've been given\nSo I'm never gonna dance again\nThe way I danced with you\n\nTime can never mend\nThe careless whispers of a good friend\nTo the heart and mind\nIgnorance is kind\nThere's no comfort in the truth\nPain is all you'll find\n\nI'm never gonna dance again\nGuilty feet have got no rhythm\nThough it's easy to pretend\nI know you're not a fool\n\nShould've known better than to cheat a friend\nAnd waste this chance that I've been given\nSo I'm never gonna dance again\nThe way I danced with you\n\nNever without your love\n\nTonight the music seems so loud\nI wish that we could lose this crowd\nMaybe it's better this 

In [88]:
test = processing_pipeline(test)

In [91]:
for x in range(len(test)):
    print(test.iloc[x]["Lyrics"])

['feel', 'unsure', 'hand', 'lead', 'dance', 'floor', 'music', 'die', 'eye', 'call', 'mind', 'silver', 'screen', 'sad', 'goodbye', 'go', 'to', 'dance', 'guilty', 'foot', 'get', 'rhythm', 'easy', 'pretend', 'know', 'fool', 'know', 'well', 'cheat', 'friend', 'waste', 'chance', 'give', 'go', 'to', 'dance', 'way', 'dance', 'Time', 'mend', 'careless', 'whisper', 'good', 'friend', 'heart', 'mind', 'Ignorance', 'kind', 'comfort', 'truth', 'pain', 'find', 'go', 'to', 'dance', 'guilty', 'foot', 'get', 'rhythm', 'easy', 'pretend', 'know', 'fool', 'know', 'well', 'cheat', 'friend', 'waste', 'chance', 'give', 'go', 'to', 'dance', 'way', 'dance', 'love', 'tonight', 'music', 'loud', 'wish', 'lose', 'crowd', 'maybe', 'well', 'way', 'hurt', 'thing', 'want', 'good', 'live', 'dance', 'forever', 'go', 'to', 'dance', 'stay', 'go', 'to', 'dance', 'guilty', 'foot', 'get', 'rhythm', 'easy', 'pretend', 'know', 'fool', 'know', 'well', 'cheat', 'friend', 'waste', 'chance', 'give', 'go', 'to', 'dance', 'way', 'da

In [7]:
# todo: vocab size bestimmen 
# todo: padding funktion einbauen 
# todo: testen mit labeled data

model = Sequential([
    Embedding(vocab_size, 8, input_length=max_length),
   Conv1D(128, 5, activation='relu'),
    GlobalMaxPooling1D(),
  Dense(10, activation='relu'),
  Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

history = model.fit(x_train, y_train, epochs=20, validation_data=(x_test, y_test))

NameError: name 'vocab_size' is not defined

In [81]:
test = ["abc"] * 5
test2 = ["cde", "efg"]

In [83]:
test2 + test


['cde', 'efg', 'abc', 'abc', 'abc', 'abc', 'abc']

In [78]:
print(a)

None
